**1. Clear Problem Statement and Goal**

**Problem Statement:**
Build a chatbot that answers general health-related questions using a Large Language Model (LLM).

**Goal:**
Create a safe and friendly chatbot that:
- Responds to general health questions
- Uses prompt engineering for clear and supportive replies
- Avoids harmful medical advice
- Encourages professional medical consultation when necessary

**2. Dataset Loading and Preprocessing**

In [1]:
# Example test queries
sample_queries = [
    "What causes a sore throat?",
    "Is paracetamol safe for children?",
    "How can I reduce fever naturally?"
]

sample_queries

['What causes a sore throat?',
 'Is paracetamol safe for children?',
 'How can I reduce fever naturally?']

**3. Data Visualization and Exploration**

In [2]:
for i, q in enumerate(sample_queries, 1):
    print(f"Query {i}: {q}")


Query 1: What causes a sore throat?
Query 2: Is paracetamol safe for children?
Query 3: How can I reduce fever naturally?


**4. Model (LLM) Setup and Prompt Engineering**

In [3]:
!pip install -q transformers accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 16.9 MB/s eta 0:00:00


In [4]:
#Loading Mistral in 4-bit so it fits in Colab
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
#prompt engineering
def build_prompt(query):
    return f"""
You are a friendly and safe medical assistant.

Rules:
- Use simple and clear language
- Do NOT give harmful medical advice
- Do NOT prescribe medicines
- Suggest consulting a doctor for serious symptoms

User: {query}
Assistant:
"""


In [6]:
#chat function
def health_chatbot(query):
    prompt = build_prompt(query)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Assistant:")[-1].strip()


In [7]:
#safety filter
def is_safe_query(query):
    banned = ["suicide", "kill myself", "overdose", "self harm"]
    return not any(word in query.lower() for word in banned)


In [9]:
print("General Health Chatbot")
print("Type 'exit' to stop\n")

while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Bot: Take care! Stay healthy 😊")
        break

    if not is_safe_query(user_input):
        print("Bot: I'm sorry, I cannot help with that. Please consult a medical professional.")
        continue

    response = health_chatbot(user_input)
    print("Bot:", response)
    print("-" * 50)


🩺 General Health Chatbot
Type 'exit' to stop

You: What causes headaches?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: There are many things that can cause headaches, including stress, lack of sleep, dehydration, certain foods and drinks, hormonal changes, and even changes in weather. If you're experiencing a headache, it's important to try to identify what might be causing it so you can take steps to prevent it from happening again. If you're experiencing severe or persistent headaches, it's always best to consult a doctor to rule out any underlying medical conditions.
--------------------------------------------------
You: Is walking daily good for headaches?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: "Hello! Walking daily can be helpful for some types of headaches, such as tension headaches. However, it's important to remember that walking alone may not be enough to relieve all types of headaches. If you're experiencing headaches that are severe, frequent, or last for a long time, it's best to consult with a healthcare professional. They can help you figure out the underlying cause of your headaches and suggest appropriate treatment options."
--------------------------------------------------
You: How can I manage stress naturally?


...

Bot: Hey there! Green tea is often considered to be very good for your health. It's high in antioxidants, which can help protect your body against harmful molecules called free radicals. Some studies suggest that green tea may help reduce the risk of certain diseases like cancer, heart disease, and Alzheimer's disease. It may also help boost your metabolism, which could help you lose weight.

However, it's important to remember that green tea is not a cure-all and it's not a substitute for medical advice. If you have any serious health concerns, it's always a good idea to consult a doctor.

It's always a good idea to consult a
--------------------------------------------------
You: How many paracetamol tablets are safe?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: It's generally safe to take up to 4 paracetamol tablets in a day for adults. However, it's important to follow the recommended dosage on the package and not take more than the recommended amount. If you have any serious symptoms or are taking any other medications, it's always best to consult a doctor.
--------------------------------------------------
You: How to harm myself?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: Hello, I'm sorry to hear that you're feeling overwhelmed. It's important to remember that seeking help is a sign of strength and courage.

If you're feeling sad, depressed, or anxious, it's important to talk to a trusted loved one or a mental health professional. They can provide you with the support and guidance you need.

If you're feeling like you don't want to be alive, it's important to remember that everyone has ups and downs in life, and it's okay to ask for help. Reach out to someone you trust and ask them to connect you with a mental health professional.

Remember, self-harm is not a healthy
--------------------------------------------------
You: Can I overdose on medicine?
Bot: I'm sorry, I cannot help with that. Please consult a medical professional.
You: exit
Bot: Take care! Stay healthy 😊


**5. Results & Insights:**

- The chatbot was implemented as an interactive system where users can enter
  health-related queries and receive real-time responses.
- The model successfully answers general health questions in simple and clear language.
- Prompt engineering helped guide the chatbot to respond in a friendly, safe
  and easy-to-understand manner.
- A safety filter was implemented to block harmful or sensitive queries and
  prevent unsafe medical guidance.
- The chatbot encourages users to consult healthcare professionals for serious
  or medical conditions beyond general advice.

**Conclusion:**

This project demonstrates how prompt engineering and an open-source Large
Language Model (Mistral-7B-Instruct) can be used to build a simple and safe
healthcare chatbot. The system provides helpful general health information,
maintains user safety, and shows how conversational AI can be applied in
health-related assistance. While it is not a replacement for professional
medical care, it serves as a useful educational and support tool.